# I. Creates the ACSF structures and basic analysis of dataset

In [ ]:
!pip install ase

In [1]:
import ase as ase
import pandas as pd
import numpy as np
import time, copy

In [2]:
import os
os.getcwd() 

'C:\\Users\\Kanishk\\Jupyter Notebook files\\molecular prediction kaggle'

In [9]:
os.path.dirname(os.getcwd())

'C:\\Users\\Kanishk\\Jupyter Notebook files'

In [10]:
path = os.path.join(os.getcwd(), 'champs-scalar-coupling')
path

'C:\\Users\\Kanishk\\Jupyter Notebook files\\molecular prediction kaggle\\champs-scalar-coupling'

In [11]:
file_folder = path
train = pd.read_csv(f'{file_folder}/train.csv')
test = pd.read_csv(f'{file_folder}/test.csv')
structures = pd.read_csv(f'{file_folder}/structures.csv')
dipole = pd.read_csv(f'{file_folder}/dipole_moments.csv')
mulliken = pd.read_csv(f'{file_folder}/mulliken_charges.csv')

In [12]:
magnetic_tensors = pd.read_csv(f'{file_folder}/magnetic_shielding_tensors.csv')
PE = pd.read_csv(f'{file_folder}/potential_energy.csv')

Below from: [dataframe reducer](https://www.kaggle.com/todnewman/keras-neural-net-for-champs)

In [17]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

print(train.shape, test.shape, structures.shape)
df_train = reduce_mem_usage(train)
df_test = reduce_mem_usage(test)
df_structures = reduce_mem_usage(structures)
df_dipole = reduce_mem_usage(dipole)
df_mulliken = reduce_mem_usage(mulliken)
df_PE = reduce_mem_usage(PE)
df_magnetic_tensors  = reduce_mem_usage(magnetic_tensors)
print(train.shape, df_test.shape)

(4658147, 6) (2505542, 5) (2358657, 6)
Mem. usage decreased to 106.62 Mb (50.0% reduction)
Mem. usage decreased to 52.57 Mb (45.0% reduction)
Mem. usage decreased to 51.74 Mb (52.1% reduction)
Mem. usage decreased to  1.13 Mb (56.2% reduction)
Mem. usage decreased to 16.09 Mb (54.2% reduction)
Mem. usage decreased to  0.81 Mb (0.0% reduction)
Mem. usage decreased to 27.00 Mb (0.0% reduction)
(4658147, 6) (2505542, 5)


In [34]:
import psutil
print(psutil.cpu_percent())
print(psutil.virtual_memory())  # physical memory usage
print('memory % used:', psutil.virtual_memory()[2])

15.1
svmem(total=17026260992, available=8380010496, percent=50.8, used=8646250496, free=8380010496)
memory % used: 50.8


In [19]:
train_molecules = len( df_train['molecule_name'].unique() )
test_molecules = len( df_test['molecule_name'].unique() )

print(f'There are {train_molecules} unique molecules in train set and {len(train)} total atoms')
print(f'There are {test_molecules} unique molecules in test set and {len(test)} total atoms')

There are 85003 unique molecules in train set and 4658147 total atoms
There are 45772 unique molecules in test set and 2505542 total atoms


In [20]:
assert len(df_structures['molecule_name'].unique()) == len( df_train['molecule_name'].unique() ) + len( df_test['molecule_name'].unique() )
print('Structures available for all molecules.')

Structures available for all molecules.


In [21]:
len(dipole) 

85003

The dipole moments and Potential Energy data is available for all molecules in train set.


**Molecule's Structure Property**

The properties of molecules provided is dependent *only* on their spatial (i.e. X, Y and Z coordinate) location. However, a general molecule can be oriented in any direction. Thus, there is a concept of 
**[Atom Centred Symmetry Functions (ACSF)](https://samabilino.wordpress.com/2018/07/07/atom-centred-symmetry-functions/)**
which helps describing the properties of the molecules using their exact locations.

The rest of this notebook will convert the structure data into ACSF features. Here onwards, the continuing notebooks will use `acsf_structures.csv` file having extracted the properties from `structures.csv`

In [10]:
#For all ACSF functions: rcut = 10.0
rcut = 10.0

#G2 - eta/Rs couples:
g2_params = [[1, 2], [0.1, 2], [0.01, 2],
           [1, 6], [0.1, 6], [0.01, 6]]

#G4 - eta/ksi/lambda triplets:
g4_params = [[1, 4,  1], [0.1, 4,  1], [0.01, 4,  1], 
           [1, 4, -1], [0.1, 4, -1], [0.01, 4, -1]]

g3_params = None
g5_params = None

In [33]:
df_structures_temp = df_structures.copy() 


,molecule_name,atom_index,atom,x,y,z
0,dsgdb9nsd_000001,0,C,-0.012695,1.085938,0.008003
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976
2,dsgdb9nsd_000001,2,H,1.011719,1.463867,0.000277
3,dsgdb9nsd_000001,3,H,-0.541016,1.447266,-0.876465
4,dsgdb9nsd_000001,4,H,-0.523926,1.437500,0.906250


In [42]:
for axes in ['x', 'y', 'z']:
    df_structures_temp.pop(axes) 
df_structures_temp.head()

,molecule_name,atom_index,atom
0,dsgdb9nsd_000001,0,C
1,dsgdb9nsd_000001,1,H
2,dsgdb9nsd_000001,2,H
3,dsgdb9nsd_000001,3,H
4,dsgdb9nsd_000001,4,H


In [22]:
# structures_idx = structures.set_index('molecule_name')
cart_coord = df_structures[['x','y','z']].values
len(cart_coord)

2358657

Grouping all the unique molecule's from the atom list

In [23]:
ss = structures.groupby('molecule_name').size()
ss = ss.cumsum()
ss.head()

molecule_name
dsgdb9nsd_000001     5
dsgdb9nsd_000002     9
dsgdb9nsd_000003    12
dsgdb9nsd_000004    16
dsgdb9nsd_000005    19
dtype: int64

In [43]:
ssx = np.zeros(len(ss) + 1, 'int')
ssx[1:] = ss
ssx

array([      0,       5,       9, ..., 2358623, 2358641, 2358657])

Computing distantces for the different molecules

In [25]:
def get_fast_dist_matrix(xyz, ssx, molecule_id):
    start_molecule, end_molecule = ssx[molecule_id], ssx[molecule_id+1]
    locs = xyz[start_molecule:end_molecule]    
    num_atoms = end_molecule - start_molecule
    loc_tile = np.tile(locs.T, (num_atoms,1,1))
    dist_mat = np.sqrt(((loc_tile - loc_tile.T)**2).sum(axis=1))
    return dist_mat

molecule_id = 0
molecule = ss.index[molecule_id]
print(molecule)
get_fast_dist_matrix(cart_coord, ssx, molecule_id)

dsgdb9nsd_000001


array([[0.   , 1.092, 1.092, 1.092, 1.092],
       [1.092, 0.   , 1.783, 1.782, 1.783],
       [1.092, 1.783, 0.   , 1.783, 1.782],
       [1.092, 1.782, 1.783, 0.   , 1.783],
       [1.092, 1.783, 1.782, 1.783, 0.   ]], dtype=float16)

In [30]:
structures['atom'][ssx[molecule_id]: ssx[molecule_id+1]]

0    C
1    H
2    H
3    H
4    H
Name: atom, dtype: object

In [44]:
# Radial functions
def fc(Rij, Rc):
    rij = np.where(Rij < Rc, Rij, Rc)
    val = 0.5 * ( np.cos(np.pi * rij/Rc) +1 ) #if Rij < Rc else 0
    return val

In [45]:
def G_rad1(Rij, Rc):                                           # 1st order Radial functions
     return fc(Rij, Rc).sum()

def G_rad2(Rij, eta, mu, Rc):                                           # 2nd order Radial functions
    return np.sum( np.exp(-eta*(Rij-mu)**2) * fc(Rij, Rc) )

def get_cos(ord_i, ord_j, ord_k):
    L1 = ord_j - ord_i
    L2 = ord_k - ord_i
    cos_theta = np.dot(L1, L2) / ( np.linalg.norm(L1) * np.linalg.norm(L2) )
    return cos_theta
    
    
def G_ang4(Rij, Rjk, Rki, Rc, xyz_i, xyz_j, xyz_k, eta, mu, xi, lam):     
    # i = 1 or 2 Radial functions
    f_pdt = G_rad1(Rij, Rc) * G_rad1(Rjk, Rc) * G_rad1(Rki, Rc)            # Doesn't contain 2^(1-xi) term
    if f_pdt == 0:
        return f_pdt
    
    else:
        exp_term = np.exp( -eta*((Rij-mu)**2 + (Rki-mu)**2 + (Rjk-mu)**2 ))
        
        return exp_term * f_pdt * (1 + lam * get_cos(xyz_i, xyz_j, xyz_k))**xi

In [88]:
params = {'Rc' : 2.2, 
         'lam' : [1, -1],
         'xi' : [1, 8, 16],
         'eta' : [0.01, 0.1, 1],
         'mu' : [2, 6]}

def make_headers(param):
    df_structures_temp['g1'] = ''

    for mu in param['mu']:
        for eta in param['eta']:
            dic = 'g2_' + str(mu) + '_' + str(eta)
            df_structures_temp[dic] = ''

    for mu in param['mu']:
        for eta in param['eta']:
            for lam in param['lam']:
                for xi in param['xi']:
                    dic = 'g4_' + str(mu) + '_' + str(eta) + '_' + str(lam) + '_' + str(xi)
                    df_structures_temp[dic] = ''

make_headers(param)

In [93]:
df_structures_temp.head(7)

,molecule_name,atom_index,atom,g1,g2_2_0.01,g2_2_0.1,g2_2_1,g2_6_0.01,g2_6_0.1,g2_6_1,...,g4_6_0.1_1_16,g4_6_0.1_-1_1,g4_6_0.1_-1_8,g4_6_0.1_-1_16,g4_6_1_1_1,g4_6_1_1_8,g4_6_1_1_16,g4_6_1_-1_1,g4_6_1_-1_8,g4_6_1_-1_16
0,dsgdb9nsd_000001,0,C,,,,,,,,...,,,,,,,,,,
1,dsgdb9nsd_000001,1,H,,,,,,,,...,,,,,,,,,,
2,dsgdb9nsd_000001,2,H,,,,,,,,...,,,,,,,,,,
3,dsgdb9nsd_000001,3,H,,,,,,,,...,,,,,,,,,,
4,dsgdb9nsd_000001,4,H,,,,,,,,...,,,,,,,,,,
5,dsgdb9nsd_000002,0,N,,,,,,,,...,,,,,,,,,,
6,dsgdb9nsd_000002,1,H,,,,,,,,...,,,,,,,,,,


The `make_molecule_data` function below will help us in iteratively creating the data.

**Note:** The ssx term is used to segment our molecule data since the below computations take a lot of time (14 hours * 10 times on my laptop) and altering it can help us in creating partial dataframes that can be later merged together

In [322]:
def finite_val(val, lt=1e-10):
    return val if val > lt else 0

def make_molecule_data(df, cart_coord, ssx, molecule_id, params):
    D_Mx = get_fast_dist_matrix(cart_coord, ssx, molecule_id)  
    xyz = cart_coord[ssx[molecule_id]: ssx[molecule_id+1]]
    n_atoms = len(xyz)
    indices = np.arange(n_atoms)        # The indices in for loop
    
    # Radial functions
    for i in indices:
        atom_ind = ssx[molecule_id]+i
#         print('Atom index=', ssx[molecule_id]+i)
        
        Di = D_Mx[i]         # All the distances
        di = Di[np.nonzero(Di > 0)]   # All the distances except from the atom itself
        
        # G1
        Rc = params['Rc']
        g_1 = G_rad1(di, Rc)
        
        df.at[atom_ind, 'g1'] = finite_val(g_1)

        # G2
        for eta in params['eta']:
            for mu in params['mu']:

                g_2 = G_rad2(di, eta, mu, Rc)
                col = 'g2_' + str(mu) + '_' + str(eta)
                df.at[atom_ind, col] = finite_val(g_2)
        
        # G4
                j_indices = indices[np.nonzero(indices != i)]        

                for lam in params['lam']:
                    for xi in params['xi']:
                        g_4 = 0                                            # 2**(1-xi)

                        for j in j_indices:
                            k_indices = j_indices[np.nonzero(j_indices != j)]        

                            for k in k_indices:
                                g_4 += G_ang4( D_Mx[i,j], D_Mx[j,k], D_Mx[i,k], Rc, xyz[i], xyz[j], xyz[k], eta, mu, xi, lam)

                        g_4 *= 2**(1-xi)
                        col = 'g4_' + str(mu) + '_' + str(eta) + '_' + str(lam) + '_' + str(xi)
                        df.at[atom_ind, col] = finite_val(g_4)

    return df

molecule_id = 0
ca = make_molecule_data(df_structures_temp, cart_coord, ssx, molecule_id, params).iloc[0]

In [323]:
temp_df = df_structures.copy()
temp_df.pop('x')
temp_df.pop('y')
temp_df.pop('z')
temp_df.head()

,molecule_name,atom_index,atom
0,dsgdb9nsd_000001,0,C
1,dsgdb9nsd_000001,1,H
2,dsgdb9nsd_000001,2,H
3,dsgdb9nsd_000001,3,H
4,dsgdb9nsd_000001,4,H


In [317]:
temp_df = make_molecule_data(temp_df, cart_coord, ssx, 6, params)
temp_df.head(60)

[[0.     1.416  1.1    1.101  1.094  1.94  ]
 [1.416  0.     2.102  2.102  2.025  0.9614]
 [1.1    2.102  0.     1.783  1.774  2.354 ]
 [1.101  2.102  1.783  0.     1.775  2.354 ]
 [1.094  2.025  1.774  1.775  0.     2.82  ]
 [1.94   0.9614 2.354  2.354  2.82   0.    ]]
C
1.8193359375
O
0.9052734375
H
0.6796875
H
0.67919921875
H
0.697265625
H
0.6318359375


,molecule_name,atom_index,atom,g1,g2_2_0.01,g4_2_0.01_1_1,g4_2_0.01_1_8,g4_2_0.01_1_16,g4_2_0.01_-1_1,g4_2_0.01_-1_8,...,g4_2_1_-1_1,g4_2_1_-1_8,g4_2_1_-1_16,g2_6_1,g4_6_1_1_1,g4_6_1_1_8,g4_6_1_1_16,g4_6_1_-1_1,g4_6_1_-1_8,g4_6_1_-1_16
0,dsgdb9nsd_000001,0,C,2.021484,2.003906,0.173329,0.000079,1.210209e-08,0.346595,2.027678e-02,...,0.064607,3.779668e-03,0.000147,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,dsgdb9nsd_000001,1,H,0.763672,0.759277,0.241837,0.121097,5.577466e-02,0.025776,1.177461e-07,...,0.006110,1.014671e-07,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,dsgdb9nsd_000001,2,H,0.763672,0.759277,0.241860,0.121185,5.585518e-02,0.025753,1.171310e-07,...,0.006105,1.009386e-07,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,dsgdb9nsd_000001,3,H,0.763672,0.759277,0.241837,0.121097,5.577466e-02,0.025776,1.177461e-07,...,0.006110,1.014671e-07,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,dsgdb9nsd_000001,4,H,0.763672,0.759277,0.241837,0.121097,5.577466e-02,0.025776,1.177461e-07,...,0.006110,1.014671e-07,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,dsgdb9nsd_000002,0,N,1.676758,1.660156,0.219180,0.000196,6.454238e-08,0.378132,1.540790e-02,...,0.048321,1.968940e-03,0.000051,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,dsgdb9nsd_000002,1,H,0.883789,0.877930,0.370556,0.169780,7.105738e-02,0.045044,2.677617e-07,...,0.007986,1.714190e-07,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,dsgdb9nsd_000002,2,H,0.883789,0.877930,0.370342,0.169682,7.101512e-02,0.045015,2.657133e-07,...,0.007985,1.700906e-07,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,dsgdb9nsd_000002,3,H,0.883301,0.877441,0.370048,0.169706,7.110154e-02,0.044923,2.636319e-07,...,0.007968,1.687644e-07,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,dsgdb9nsd_000003,0,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [108]:
# Making ACSF structures for 1st molecule
df_structures_temp.head(10)

,molecule_name,atom_index,atom,g1,g2_2_0.01,g2_2_0.1,g2_2_1,g2_6_0.01,g2_6_0.1,g2_6_1,...,g4_6_0.1_1_16,g4_6_0.1_-1_1,g4_6_0.1_-1_8,g4_6_0.1_-1_16,g4_6_1_1_1,g4_6_1_1_8,g4_6_1_1_16,g4_6_1_-1_1,g4_6_1_-1_8,g4_6_1_-1_16
0,dsgdb9nsd_000001,0,C,2.02148,2.00391,1.86133,0.88623,1.58887,0.182251,0,...,1.68037e-11,0.000481245,2.81542e-05,1.098e-06,3.96268e-30,1.81349e-33,2.7668e-37,7.92392e-30,4.63571e-31,1.80791e-32
1,dsgdb9nsd_000001,1,H,0.763672,0.759277,0.722168,0.467773,0.613281,0.0891113,0,...,7.77067e-05,4.23612e-05,5.63689e-10,8.55132e-15,3.88006e-26,5.18523e-27,5.20558e-28,1.29267e-26,7.87157e-31,1.19801e-35
2,dsgdb9nsd_000001,2,H,0.763672,0.759277,0.722168,0.467773,0.613281,0.0891113,0,...,7.78194e-05,4.23258e-05,5.60753e-10,8.46243e-15,3.88091e-26,5.19423e-27,5.22364e-28,1.29183e-26,7.83061e-31,1.18555e-35
3,dsgdb9nsd_000001,3,H,0.763672,0.759277,0.722168,0.467773,0.613281,0.0891113,0,...,7.77067e-05,4.23612e-05,5.63689e-10,8.55132e-15,3.88006e-26,5.18523e-27,5.20558e-28,1.29267e-26,7.87157e-31,1.19801e-35
4,dsgdb9nsd_000001,4,H,0.763672,0.759277,0.722168,0.467773,0.613281,0.0891113,0,...,7.77067e-05,4.23612e-05,5.63689e-10,8.55132e-15,3.88006e-26,5.18523e-27,5.20558e-28,1.29267e-26,7.87157e-31,1.19801e-35
5,dsgdb9nsd_000002,0,N,,,,,,,,...,,,,,,,,,,
6,dsgdb9nsd_000002,1,H,,,,,,,,...,,,,,,,,,,
7,dsgdb9nsd_000002,2,H,,,,,,,,...,,,,,,,,,,
8,dsgdb9nsd_000002,3,H,,,,,,,,...,,,,,,,,,,
9,dsgdb9nsd_000003,0,O,,,,,,,,...,,,,,,,,,,


In [151]:
# Lesser number of molecules than last index
arr = df_structures.molecule_name.unique()

In [152]:
arr_l=[]
for array in arr:
    arr_no = int(array[-6:])
    
    arr_l.append(arr_no)

In [333]:
def make_data(df_structures_temp, cart_coord, ssx, molecules, params):
    df = df_structures_temp
    for molecule in molecules:         #range(first_molecule, last_molecule):
        if molecule%100 == 0:
            print(f'Trained {molecule}')
            
        df = make_molecule_data(df, cart_coord, ssx, molecule, params)
    
    return df
        
new_ = make_data(new_, cart_coord, ssx,np.arange(10), params)        

Trained 0


In [196]:
df_structures_temp.head()

,molecule_name,atom_index,atom,g1,g2_2_0.01,g2_2_0.1,g2_2_1,g2_6_0.01,g2_6_0.1,g2_6_1,...,g4_6_0.1_1_16,g4_6_0.1_-1_1,g4_6_0.1_-1_8,g4_6_0.1_-1_16,g4_6_1_1_1,g4_6_1_1_8,g4_6_1_1_16,g4_6_1_-1_1,g4_6_1_-1_8,g4_6_1_-1_16
0,dsgdb9nsd_000001,0,C,2.02148,2.00391,1.86133,0.88623,1.58887,0.182251,0,...,0,0.000481245,2.81542e-05,1.098e-06,0,0,0,0,0,0
1,dsgdb9nsd_000001,1,H,0.763672,0.759277,0.722168,0.467773,0.613281,0.0891113,0,...,7.77067e-05,4.23612e-05,5.63689e-10,0,0,0,0,0,0,0
2,dsgdb9nsd_000001,2,H,0.763672,0.759277,0.722168,0.467773,0.613281,0.0891113,0,...,7.78194e-05,4.23258e-05,5.60753e-10,0,0,0,0,0,0,0
3,dsgdb9nsd_000001,3,H,0.763672,0.759277,0.722168,0.467773,0.613281,0.0891113,0,...,7.77067e-05,4.23612e-05,5.63689e-10,0,0,0,0,0,0,0
4,dsgdb9nsd_000001,4,H,0.763672,0.759277,0.722168,0.467773,0.613281,0.0891113,0,...,7.77067e-05,4.23612e-05,5.63689e-10,0,0,0,0,0,0,0


Convert the dataframe to a pickle file to be loaded later and merged to create a single `acsf_structures.csv` file that will be used for modeling

In [242]:
new_.to_pickle('df1.pkl')

In [243]:
df = pd.read_pickle('df1.pkl')